In [ ]:
# Setting seeds for reproducibility
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import json
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import ADASYN, SMOTE
import pandas as pd
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
data_file_path = "../../Features/PCFs/files_for_ml/protein_props.json"
druggable_proteins_file_path = "../../DrugBank/druggable_proteins.txt"
approved_druggable_proteins_file_path = "../../DrugBank/approved_druggable_proteins.txt"

with open(data_file_path, 'r') as f:
    protein_data = json.load(f)

print("Total number of uniprot human verified proteins:", len(protein_data))

# Extracting list of druggable and approved druggable proteins
with open(druggable_proteins_file_path, 'r') as f:
    druggable_proteins = f.read().splitlines()

with open(approved_druggable_proteins_file_path, 'r') as f:
    approved_druggable_proteins = f.read().splitlines()

print("Number of druggable proteins:", len(druggable_proteins))
print("Number of approved druggable proteins:", len(approved_druggable_proteins))


# Fetching feature data for all proteins
properties = (pd.read_json("../../Features/PCFs/files_for_ml/protein_props.json")).transpose()
is_druggable = [1 if i in druggable_proteins else 0 for i in properties.index]
is_approved_druggable = [1 if i in approved_druggable_proteins else 0 for i in properties.index]

properties["is_druggable"] = is_druggable
properties["is_approved_druggable"] = is_approved_druggable

PCP_properties = properties.copy()
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
amino_acid_percent = {i:[] for i in amino_acids}
for i in PCP_properties['Amino Acid Percent']:
  for aa in amino_acids:
    amino_acid_percent[aa].append(i[aa])
for aa in amino_acids:
  PCP_properties = pd.concat([PCP_properties, pd.Series(amino_acid_percent[aa], index = PCP_properties.index, name = f"Amino Acid Percent {aa}")], axis = 1)

PCP_properties[f"Molar Extinction Coefficient 1"] = pd.Series([x[0] for x in PCP_properties['Molar Extinction Coefficient']], index = PCP_properties.index)
PCP_properties[f"Molar Extinction Coefficient 2"] = pd.Series([x[1] for x in PCP_properties['Molar Extinction Coefficient']], index = PCP_properties.index)

PCP_properties[f"Secondary Structure helix"] = pd.Series([x[0] for x in PCP_properties['Secondary Structure']], index = PCP_properties.index)
PCP_properties[f"Secondary Structure turn"] = pd.Series([x[1] for x in PCP_properties['Secondary Structure']], index = PCP_properties.index)
PCP_properties[f"Secondary Structure sheet"] = pd.Series([x[2] for x in PCP_properties['Secondary Structure']], index = PCP_properties.index)

PCP_properties.drop(columns = ['Amino Acid Count','Amino Acid Percent',"Molar Extinction Coefficient","Flexibility","Secondary Structure",'Sequence'], inplace = True)
PCP_properties['Sequence Length'] = PCP_properties['Sequence Length'].astype(int)
PCP_properties[['Molecular Weight', 'GRAVY', 'Isoelectric Point', 'Instability Index', 'Aromaticity', 'Charge at 7']] = PCP_properties[['Molecular Weight', 'GRAVY', 'Isoelectric Point', 'Instability Index', 'Aromaticity', 'Charge at 7']].astype(float)

with open("/content/drive/MyDrive/protein_props/features/gdpc_encodings.json", 'r') as file:
    data = json.load(file)
gpdc_encodings = pd.DataFrame(data).transpose()

ppi = pd.read_json("../../Features/PPIs/files_for_ml/ppi.json").transpose()
ppi_network = pd.read_csv("../../Features/PPIs/files_for_ml/ppi_network_properties.csv")
ppi_network.index = ppi_network['Unnamed: 0']
ppi_network.drop(columns = ['Unnamed: 0'], inplace = True)
ppi = pd.concat([ppi, ppi_network], axis = 1)

glycolisation = pd.read_csv("../../Features/PTMs/files_for_ml/glycosylation.csv")
glycolisation.index = glycolisation['Unnamed: 0']
glycolisation.drop(columns = ['Unnamed: 0'], inplace = True)
ptm = pd.read_csv("../../Features/PTMs/files_for_ml/PTM_counts.csv")
ptm.index = ptm["Unnamed: 0"]
ptm.drop(columns = ['Unnamed: 0'], inplace = True)
ptm_counts = pd.concat([ptm, glycolisation], axis = 1)

with open("../../Features/SCL/files_for_ml/subcellular_locations2.json", 'r') as file:
    data = json.load(file)
unique_groups = set()
for entry in data.values():
    if "general" in entry:
        for general_entry in entry["general"]:
            if "group" in general_entry: unique_groups.add(general_entry["group"])

unique_groups_list = list(unique_groups)

rows = []
for protein_id in PCP_properties.index:
    row = {group: 0 for group in unique_groups_list}
    if protein_id in data:
        for entry in data[protein_id].get("general", []):
            if "group" in entry and entry["group"] in unique_groups:
                row[entry["group"]] = 1
    row["protein_id"] = protein_id
    rows.append(row)

subcellular_data = pd.DataFrame(rows).set_index("protein_id")

domains = pd.read_csv("../../Features/Domains/files_for_ml/data_top20.csv")
domains.index = domains['Unnamed: 0']
domains.drop(columns = ['Unnamed: 0'], inplace = True)

flexibility = pd.read_csv("../../Features/PCFs/files_for_ml/flexibility_properties.csv")
flexibility.index = flexibility['Unnamed: 0']
flexibility.drop(columns = ['Unnamed: 0'], inplace = True)

latent_data = pd.read_csv("../../Features/Latents/files_for_ml/latent_values.csv").transpose()
latent_data.columns = [f"Latent_Value_{i+1}" for i in latent_data.columns]
final_data = pd.concat([PCP_properties,gpdc_encodings, ptm_counts, ppi, subcellular_data, domains, flexibility, latent_data], axis = 1).dropna()
features_list = final_data.columns
features_list = features_list.drop(['is_druggable','is_approved_druggable'])
features_list = list(features_list)
print(features_list)
print(len(features_list))


Total number of uniprot human verified proteins: 20434
Number of druggable proteins: 3345
Number of approved druggable proteins: 2652
['Sequence Length', 'Molecular Weight', 'GRAVY', 'Isoelectric Point', 'Instability Index', 'Aromaticity', 'Charge at 7', 'Amino Acid Percent A', 'Amino Acid Percent C', 'Amino Acid Percent D', 'Amino Acid Percent E', 'Amino Acid Percent F', 'Amino Acid Percent G', 'Amino Acid Percent H', 'Amino Acid Percent I', 'Amino Acid Percent K', 'Amino Acid Percent L', 'Amino Acid Percent M', 'Amino Acid Percent N', 'Amino Acid Percent P', 'Amino Acid Percent Q', 'Amino Acid Percent R', 'Amino Acid Percent S', 'Amino Acid Percent T', 'Amino Acid Percent V', 'Amino Acid Percent W', 'Amino Acid Percent Y', 'Molar Extinction Coefficient 1', 'Molar Extinction Coefficient 2', 'Secondary Structure helix', 'Secondary Structure turn', 'Secondary Structure sheet', 'aliphatic_aliphatic', 'aliphatic_positive', 'aliphatic_negative', 'aliphatic_uncharged', 'aliphatic_aromatic',

In [ ]:
# Train Test Splitting
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import ADASYN, SMOTE

def get_data(x_sample, y_sample):
  return np.array(x_sample), np.array(y_sample)

def data_splitting(x_sample, y_sample, mode="default", scaler="none", class_size=600):
  druggable_indices = (y_sample == 1)  # Assuming 1 represents druggable
  non_druggable_indices = (y_sample == 0)  # Assuming 0 represents non-druggable

  druggable_X = x_sample[druggable_indices]
  druggable_y = y_sample[druggable_indices]

  non_druggable_X = x_sample[non_druggable_indices]
  non_druggable_y = y_sample[non_druggable_indices]

  druggable_X_remaining, druggable_X_test, druggable_y_remaining, druggable_y_test = train_test_split(druggable_X, druggable_y, test_size=class_size, random_state=123)
  non_druggable_X_remaining, non_druggable_X_test, non_druggable_y_remaining, non_druggable_y_test = train_test_split(non_druggable_X, non_druggable_y, test_size= class_size, random_state=123)

  X_test = pd.concat((druggable_X_test, non_druggable_X_test))
  y_test = pd.concat((druggable_y_test, non_druggable_y_test))
  X_train = pd.concat((druggable_X_remaining, non_druggable_X_remaining))
  y_train = pd.concat((druggable_y_remaining, non_druggable_y_remaining))
  X_train, y_train = shuffle(X_train, y_train, random_state=123)
  if mode == "default":
    pass
  elif mode == "adasyn":
    ada = ADASYN(random_state=42)
    X_train, y_train = ada.fit_resample(X_train, y_train)
  elif mode == "smote":
    smt = SMOTE(random_state=42)
    X_train, y_train = smt.fit_resample(X_train, y_train)

  if scaler == "std":
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
  elif scaler == "minmax":
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
  elif scaler == "none":
    pass

  return X_train, X_test, y_train, y_test


In [ ]:
# rem-new-data is to extract only those proteins which are either approved druggable or non-druggable
# i.e., it excludes proteins which are non-approved but druggable
new_data = final_data.copy()
new_data['new_column'] = new_data['is_druggable'] + new_data['is_approved_druggable']
rem_new_data = new_data[new_data['new_column'] != 1]
rem_new_data.shape, np.bincount(rem_new_data['new_column'])

((19585, 186), array([16949,     0,  2636]))

### Predicting Druggability Index (DI) using Partition Method

In [ ]:
X, y = rem_new_data[features_list], np.array(rem_new_data["is_approved_druggable"])
X.shape, y.shape

((19585, 183), (19585,))

In [ ]:
np.bincount(y)

array([16949,  2636])

In [ ]:
X_druggable = X[y == 1]
X_non_druggable = X[y == 0]

X_non_druggable_partitions = np.array_split(X_non_druggable, round(len(X_non_druggable)/len(X_druggable)))
print(f"Splitting into {len(X_non_druggable_partitions)} partitions")
print("Sizes of partitions")
for i, partition in enumerate(X_non_druggable_partitions):
  print(f"Partition {i}: {len(partition)}")

Splitting into 6 partitions
Sizes of partitions
Partition 0: 2825
Partition 1: 2825
Partition 2: 2825
Partition 3: 2825
Partition 4: 2825
Partition 5: 2824


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
import xgboost as xgb

xgb_models = []
for partition in X_non_druggable_partitions:
  X_combined = np.concatenate((np.array(X_druggable), np.array(partition)))
  y_combined = np.concatenate((np.ones(len(X_druggable)), np.zeros(len(partition))))
  xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
  xgb_model.fit(X_combined, y_combined)
  xgb_models.append(xgb_model)


### Druggability Index for Non Approved Druggable Proteins

In [ ]:
non_approved_druggable = new_data[new_data["new_column"] == 1]
non_approved_druggable.shape

(688, 186)

In [ ]:
X_test, y_test = get_data(non_approved_druggable[features_list], non_approved_druggable["is_approved_druggable"])
X_test.shape, y_test.shape

In [ ]:
protein_names = non_approved_druggable.index
len(protein_names)

In [ ]:
predictions = []
probabilities = []
for model in xgb_models:
  predictions.append(model.predict(X_test))
  probabilities.append(model.predict_proba(X_test)[:,1])

predictions = np.array(predictions)
probabilities = np.array(probabilities)

predictions.shape, probabilities.shape

In [ ]:
predictions = np.mean(predictions, axis=0)
predictions = np.round(predictions)
predictions.shape

In [ ]:
mean_probabilities = np.mean(probabilities, axis=0)
mean_probabilities.shape

In [ ]:
data = {
    "Protein": protein_names
}
for i,probs in enumerate(probabilities):
  data[f"Probability_Partition_{i+1}"] = probs
data["Mean_Probability"] = mean_probabilities
data["Majority_Prediction"] = predictions

df = pd.DataFrame(data)
df.set_index("Protein", inplace=True)
df.head()

In [ ]:
df.to_csv("XGB/investigational_druggability_index_xgboost_pec.csv")

### Druggability Index for Non Druggable Train Set

In [ ]:
protein_names_all, predictions_all, probabilities_all = [], None, None
for i, partition in enumerate(X_non_druggable_partitions):
  protein_names = partition.index
  predictions, probabilities = [], []
  for j, model in enumerate(xgb_models):
    if j != i:
      predictions.append(model.predict(np.array(partition)))
      probabilities.append(model.predict_proba(np.array(partition))[:,1])
  predictions, probabilities = np.array(predictions), np.array(probabilities)
  predictions = np.mean(predictions, axis=0)
  predictions = np.round(predictions)
  print(predictions.shape, probabilities.shape)

  protein_names_all.extend(protein_names)
  if predictions_all is None:
    predictions_all = predictions
    probabilities_all = probabilities
  else:
    predictions_all = np.concatenate((predictions_all, predictions))
    probabilities_all = np.concatenate((probabilities_all, probabilities), axis=1)

(2825,) (5, 2825)
(2825,) (5, 2825)
(2825,) (5, 2825)
(2825,) (5, 2825)
(2825,) (5, 2825)
(2824,) (5, 2824)


In [ ]:
predictions_all = np.array(predictions_all)
probabilities_all = np.array(probabilities_all)
mean_probabilities_all = np.mean(probabilities_all, axis=0)
len(protein_names_all), predictions_all.shape, probabilities_all.shape, mean_probabilities_all.shape

(16949, (16949,), (5, 16949), (16949,))

In [ ]:
data = {
    "Protein": protein_names_all
}
for i,probs in enumerate(probabilities_all):
  data[f"Probability_Partition_{i+1}"] = probs
data["Mean_Probability"] = mean_probabilities_all
data["Majority_Prediction"] = predictions_all

df = pd.DataFrame(data)
df.set_index("Protein", inplace=True)
df.head()

,Probability_Partition_1,Probability_Partition_2,Probability_Partition_3,Probability_Partition_4,Probability_Partition_5,Mean_Probability,Majority_Prediction
Protein,,,,,,,
A0A087X1C5,0.885494,0.900505,0.654295,0.856481,0.995312,0.858417,1.0
A0A0B4J2F0,0.669594,0.497079,0.178652,0.347567,0.929173,0.524413,0.0
A0A0B4J2F2,0.963219,0.987862,0.987098,0.934569,0.883177,0.951185,1.0
A0A0C5B5G6,0.418199,0.451553,0.794751,0.127927,0.395506,0.437587,0.0
A0A0K2S4Q6,0.672305,0.395551,0.055447,0.001300,0.095844,0.244089,0.0


In [ ]:
1 - (sum(predictions_all)/len(predictions_all))

0.6933742403681633

In [ ]:
df.to_csv("XGB/non_druggable_druggability_index_xgboost_ploec.csv")

### Druggability Index for Druggable Train Set

In [ ]:
protein_names = X_druggable.index
predictions, probabilities = [], []
for model in xgb_models:
  predictions.append(model.predict(np.array(X_druggable)))
  probabilities.append(model.predict_proba(np.array(X_druggable))[:,1])
predictions, probabilities = np.array(predictions), np.array(probabilities)
predictions = np.mean(predictions, axis=0)
predictions = np.round(predictions)
print(predictions.shape, probabilities.shape)
mean_probabilities = np.mean(probabilities, axis=0)
mean_probabilities.shape

(2636,) (6, 2636)


(2636,)

In [ ]:
data = {
    "Protein": protein_names
}
for i,probs in enumerate(probabilities):
  data[f"Probability_Partition_{i+1}"] = probs
data["Mean_Probability"] = mean_probabilities
data["Majority_Prediction"] = predictions

df = pd.DataFrame(data)
df.set_index("Protein", inplace=True)
df.head()

,Probability_Partition_1,Probability_Partition_2,Probability_Partition_3,Probability_Partition_4,Probability_Partition_5,Probability_Partition_6,Mean_Probability,Majority_Prediction
Protein,,,,,,,,
A5X5Y0,0.997124,0.997072,0.984472,0.949442,0.998169,0.999790,0.987678,1.0
C9JRZ8,0.964498,0.993043,0.983790,0.982112,0.994586,0.996707,0.985790,1.0
O00141,0.987320,0.996682,0.999509,0.967255,0.999997,0.999999,0.991794,1.0
O00142,0.986158,0.884080,0.931365,0.946131,0.923213,0.979660,0.941768,1.0
O00180,0.863223,0.959348,0.937224,0.903561,0.993643,0.999993,0.942832,1.0


In [ ]:
df.to_csv("XGB/druggable_druggability_index_xgboost_pec.csv")